# Aplicação de Machine Learning com MLflow

O **MLflow** é uma plataforma de código aberto voltada para o **ciclo de vida completo de Machine Learning (ML)**.  
Ele foi desenvolvido para ajudar equipes de ciência de dados e engenharia a **organizar, rastrear e gerenciar experimentos, modelos e pipelines** de ML de forma simples e escalável.

## Para que serve
- **Rastreamento de experimentos**: registra parâmetros, métricas, artefatos e resultados de cada execução, permitindo comparação entre modelos.  
- **Gerenciamento de modelos**: centraliza o versionamento, facilita a implantação e define estágios de modelos (ex.: *Staging*, *Production*).  
- **Reprodutibilidade**: garante que experimentos possam ser repetidos em diferentes ambientes.  
- **Escalabilidade**: integra-se com diferentes ferramentas, frameworks (TensorFlow, PyTorch, Scikit-Learn etc.) e provedores de nuvem.

## Principais funções
1. **MLflow Tracking** – rastreia execuções, parâmetros, métricas e artefatos.  
2. **MLflow Projects** – padroniza pacotes de código para reuso e reprodutibilidade.  
3. **MLflow Models** – gerencia diferentes versões de modelos, com suporte a diversos formatos.  
4. **MLflow Registry** – armazena, versiona e controla o ciclo de vida dos modelos.

## Por que as empresas utilizam
- Facilita a **colaboração** entre cientistas de dados e engenheiros.  
- Reduz a **complexidade operacional** do ciclo de vida de ML.  
- Permite **governança e controle** sobre modelos em produção.  
- Acelera o **time-to-market** de soluções baseadas em Inteligência Artificial.  

Em resumo, o MLflow se tornou uma ferramenta essencial para empresas que desejam **profissionalizar e escalar seus processos de Machine Learning**, com rastreabilidade, governança e eficiência.


In [ ]:
# Instalar bibliotecas necessárias:
%pip install pandas matplotlib seaborn sklearn 

In [31]:
# Utilidades
import os
import pandas as pd

# Scikit-learn
from sklearn import model_selection, metrics, tree, ensemble

# MLflow
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient


In [32]:
# definir pasta de trabalho
os.chdir(r"D:\OneDrive\Documentos\GitHub\portifolio\aplicacao_mlflow")

-----------------

### Exemplo simples de Machine Learning para prever churn 

Neste exemplo de aplicação de machine learning iremos prever o churn sem utilizar o mlflow e em uma segunda etapa iremos utilizar o mlflow

In [33]:
## Carregar dados
df = pd.read_csv("data/abt.csv", sep=",")

# visualizar
df

,dt_ref,id_cliente,dias_primeira_iteracao_life,dias_ultima_iteracao_life,qtd_iteracoes_life,saldo_atual_life,pontos_acum_life,pontos_neg_life,frequencia_life,pontos_acum_dia_life,...,pct_transacao_dia04_d28,pct_transacao_dia05_d28,pct_transacao_dia06_d28,pct_transacao_dia07_d28,pct_transacao_manha_d28,pct_transacao_tarde_d28,pct_transacao_noite_d28,media_dias_recorrencia_d28,mediana_dias_recorrencia_d28,flag_churn
0,2024-10-14,000ff655-fa9f-4baa-a108-47f581ec52a1,224,17,268,686,2686,-2000,28,95.928571,...,0.000000,0.000000,1.000000,0.0,1.0,0.0,0.0,NaN,NaN,1
1,2024-05-27,000ff655-fa9f-4baa-a108-47f581ec52a1,84,20,263,534,2534,-2000,25,101.360000,...,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,8.000000,8.0,0
2,2024-06-24,000ff655-fa9f-4baa-a108-47f581ec52a1,112,24,266,635,2635,-2000,27,97.592593,...,0.666667,0.000000,0.333333,0.0,1.0,0.0,0.0,2.000000,2.0,1
3,2024-04-29,000ff655-fa9f-4baa-a108-47f581ec52a1,56,3,257,1431,2431,-1000,23,105.695652,...,0.260870,0.144928,0.159420,0.0,1.0,0.0,0.0,2.272727,2.0,0
4,2024-04-01,000ff655-fa9f-4baa-a108-47f581ec52a1,28,3,188,275,1275,-1000,11,115.909091,...,0.101064,0.154255,0.409574,0.0,1.0,0.0,0.0,2.500000,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4050,2024-04-29,feb1a34d-202a-4224-a1b3-0e737b12dd4d,39,5,40,1421,1421,0,19,74.789474,...,0.171429,0.142857,0.228571,0.0,1.0,0.0,0.0,1.571429,1.0,0
4051,2024-07-23,fec72337-de7d-4f97-adba-38de8583974a,1,1,1,1,1,0,1,1.000000,...,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,NaN,NaN,1
4052,2024-06-11,ff1ceaef-650c-422b-bdc3-6984e29e7aa5,29,27,15,162,162,0,3,54.000000,...,0.200000,0.000000,0.000000,0.0,1.0,0.0,0.0,1.000000,1.0,1
4053,2024-05-14,ff1ceaef-650c-422b-bdc3-6984e29e7aa5,1,1,10,59,59,0,1,59.000000,...,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,NaN,NaN,0


In [34]:
# definir variaveis explicativas do modelo
features = df.columns[2:-1]
X = df[features]

# definir a variável dependente que queremos prever
target = "flag_churn"
y = df[target]

# Dividir os dados entre treino e teste
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

print("Taxa de resposta train:", y_train.mean())
print("Taxa de resposta test:", y_test.mean())

Taxa de resposta train: 0.4349568434032059
Taxa de resposta test: 0.4315659679408138


In [35]:
## Montar modelos de treinamento, fazer predict e ver acurácia 

# Decision Tree
clf = tree.DecisionTreeClassifier(min_samples_leaf=100, 
                                  min_samples_split=10, 
                                  random_state=42)

# Random Forest
clf = ensemble.RandomForestClassifier(n_estimators=500,
                                      min_samples_leaf=20,
                                      random_state=123)

# Treinar modelos
clf.fit(X_train, y_train)

# Realizar predict dos modelos
y_train_predict = clf.predict(X_train)
y_test_predict = clf.predict(X_test)

# verificar a acurácia dos modelos
acc_train = metrics.accuracy_score(y_train, y_train_predict)
acc_test = metrics.accuracy_score(y_test, y_test_predict)

# visualizar acurácia
print("Acurácia train:", acc_train)
print("Acurácia test:", acc_test)

2025/08/21 23:15:45 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b5710aa63d844575b2bb6c6a051085cf', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/08/21 23:15:45 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ecosi\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <http

🏃 View run indecisive-lark-799 at: http://localhost:5000/#/experiments/138472396392580619/runs/b5710aa63d844575b2bb6c6a051085cf
🧪 View experiment at: http://localhost:5000/#/experiments/138472396392580619


2025/08/21 23:15:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ecosi\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/21 23:15:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ecosi\anaconda3\Lib\site-packages\mlflow\types

Acurácia train: 0.7607891491985204
Acurácia test: 0.7114673242909988


-----------------

### Exemplo simples de Machine Learning para prever churn integrado ao MLflow

Neste exemplo vamos construir uma aplicação de **Machine Learning** para prever o **churn** (rotatividade de clientes) e, ao mesmo tempo, integrar todo o processo com o **MLflow**.

Na prática, o fluxo funciona da seguinte forma:
1. Treinamos um modelo de classificação para identificar clientes com maior risco de churn.  
2. Realizamos previsões (*predict*) sobre os dados de teste.  
3. Registramos parâmetros, métricas e artefatos diretamente na interface do **MLflow**, permitindo rastrear e comparar execuções de forma estruturada.

---------

#### Configurando o MLflow para conectar ao servidor em execução paralelo à UI

Podemos configurar o cliente do **MLflow** (no código Python ou R) para se conectar diretamente ao servidor que está sendo executado em paralelo com a interface web (UI).

Na interface do MLflow, cada experimento possui um **ID único**.  
Esse **ID** pode ser obtido clicando no ícone de informação (a letra *"i"* dentro de um círculo), localizado ao lado do nome *default* na UI do MLflow.  

Com esse ID em mãos, conseguimos indicar explicitamente em qual experimento o código deve registrar parâmetros, métricas e modelos.


In [36]:
# Aqui é o endereço onde o mlflow está rodando
mlflow.set_tracking_uri("http://127.0.0.1:5000//")

# configurar o id 
# mlflow.set_experiment(experiment_id=0)

---------

#### Criando um experimento no MLflow

> Importante: rodar o comando para criar um experimento **mais de uma vez** não sobrescreve o que já existe.

O comportamento é o seguinte:

- **Se o experimento já existir**: o MLflow apenas retorna o mesmo `experiment_id` e passa a registrar as execuções nele.  
- **Se o experimento ainda não existir**: o MLflow cria automaticamente um novo experimento com esse nome e gera um novo `experiment_id`.

E quanto aos **runs**?  
Cada execução realizada dentro de um experimento gera um **novo run**, identificado por um ID próprio.  
Isso significa que você pode ter **vários runs no mesmo experimento**, facilitando a comparação entre diferentes treinamentos (por exemplo, testando diferentes hiperparâmetros).

In [37]:
# defina o nome do experimento que iremos enviar as informações
mlflow.set_experiment("meu_experimento")

# Define uma descrição como tag do experimento
mlflow.set_experiment_tags({"description": "Experimento de churn com Random Forest e Decision Tree"})


### Para exlucir o experimento podemos fazer assim

# client = MlflowClient()

#  # Pegando experimento pelo nome
#  exp = client.get_experiment_by_name("meu_experimento")

#  # Excluindo (na verdade faz um "soft delete")
#  client.delete_experiment(exp.experiment_id)

---------

### Rodando modelos e registrando informações no MLflow

Nesta etapa vamos treinar modelos de Machine Learning e integrar todo o processo ao **MLflow**.  
O fluxo funciona assim:

1. **Iniciamos um run no MLflow** com `mlflow.start_run()`.  
   Isso cria uma execução que ficará registrada na interface (UI) para rastrear parâmetros, métricas e artefatos.  

2. **Ativamos o autolog para sklearn** (`mlflow.sklearn.autolog()`), permitindo que o MLflow capture automaticamente:
   - parâmetros utilizados no modelo,  
   - métricas de avaliação,  
   - e até o próprio artefato do modelo treinado.  

3. **Treinamos o modelo** (neste caso um `RandomForestClassifier`) sobre os dados de treino.  

4. **Geramos previsões** tanto para os dados de treino quanto de teste e calculamos as métricas de acurácia.  

5. **Registramos as métricas manualmente no MLflow** usando `mlflow.log_metrics()`.  
   Isso garante que teremos armazenados, de forma explícita, os valores de acurácia do treino e do teste.  

6. **Visualizamos as métricas no console** e também na UI do MLflow, o que facilita a comparação entre diferentes execuções.

Assim, conseguimos acompanhar o desempenho dos modelos e manter um histórico organizado dos experimentos realizados.


In [38]:
# Habilita autolog do sklearn (registra params, métricas padrão e o modelo ao .fit)
mlflow.sklearn.autolog()

def train_and_log_model(model, X_train, y_train, X_test, y_test, run_name):
    """Treina um modelo, calcula métricas e registra tudo no MLflow em um run separado."""
    with mlflow.start_run(run_name=run_name):
        # Treino
        model.fit(X_train, y_train)

        # Predições
        y_train_pred = model.predict(X_train)
        y_test_pred  = model.predict(X_test)

        # Métricas
        acc_train = metrics.accuracy_score(y_train, y_train_pred)
        acc_test  = metrics.accuracy_score(y_test,  y_test_pred)

        # Log explícito das métricas principais (além do autolog)
        mlflow.log_metrics({
            "acc_train": acc_train,
            "acc_test": acc_test
        })

        # (Opcional) tags úteis para filtrar na UI
        mlflow.set_tag("model_class", model.__class__.__name__)

    return acc_train, acc_test

# === Modelo 1: Decision Tree ===
dt = tree.DecisionTreeClassifier(
    min_samples_leaf=100,
    min_samples_split=10,
    random_state=42
)
dt_acc_train, dt_acc_test = train_and_log_model(dt, X_train, y_train, X_test, y_test, run_name="DecisionTree")

# === Modelo 2: Random Forest ===
rf = ensemble.RandomForestClassifier(
    n_estimators=500,
    min_samples_leaf=20,
    random_state=123
)
rf_acc_train, rf_acc_test = train_and_log_model(rf, X_train, y_train, X_test, y_test, run_name="RandomForest")

# Prints rápidos
print(f"[DecisionTree] acc_train={dt_acc_train:.4f} | acc_test={dt_acc_test:.4f}")
print(f"[RandomForest] acc_train={rf_acc_train:.4f} | acc_test={rf_acc_test:.4f}")


2025/08/21 23:15:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ecosi\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/21 23:15:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ecosi\anaconda3\Lib\site-packages\mlflow\types

🏃 View run DecisionTree at: http://127.0.0.1:5000/#/experiments/138472396392580619/runs/75dceb6f8163455bb3058f0f2520470f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/138472396392580619


2025/08/21 23:16:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ecosi\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/21 23:16:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ecosi\anaconda3\Lib\site-packages\mlflow\types

🏃 View run RandomForest at: http://127.0.0.1:5000/#/experiments/138472396392580619/runs/57f81241c44843b08deeb8b3afb8c8e8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/138472396392580619
[DecisionTree] acc_train=0.7284 | acc_test=0.7263
[RandomForest] acc_train=0.7608 | acc_test=0.7115


---------------

### Registrando a run no MLflow

Nesta etapa vamos registrar a execução (*run*) no **Model Registry** do MLflow.  

Na interface (UI), podemos criar um modelo no registro e associar a ele o **run** referente ao melhor modelo treinado.  
Isso garante o vínculo entre o experimento e a versão registrada do modelo, facilitando o rastreamento, a comparação entre runs e a futura implantação em produção.

**Nota:** também é possível realizar esse registro diretamente via código (usando a API do MLflow).  
No entanto, como geralmente estamos comparando métricas e avaliando modelos diretamente na UI, é comum fazer esse processo manualmente por lá.



#### Pegar o modelo treinado e registrado diretamente do mlflow

In [39]:
# Aponta para o servidor (UI rodando no localhost)
mlflow.set_tracking_uri("http://localhost:5000")

# Carrega a versão 1 do modelo registrado (este nome está na url do modelo registrado)
model = mlflow.sklearn.load_model("models:/churn model teste/1")
model

,n_estimators,500
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,20
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


#### Realizar o predict do modelo baixado do mlflow

In [40]:
# Realizar predict
y_pred = model.predict(X_test)

# Obter a probabilidades
proba = model.predict_proba(X)
proba

array([[0.24962664, 0.75037336],
       [0.47578737, 0.52421263],
       [0.31212063, 0.68787937],
       ...,
       [0.14001329, 0.85998671],
       [0.73817854, 0.26182146],
       [0.533172  , 0.466828  ]])

#### Vantagens

Uma das principais vantagens do **Model Registry** do MLflow é a facilidade de gerenciar múltiplas versões de um mesmo modelo.

Por exemplo, se tivéssemos treinado e registrado outro modelo, poderíamos consumi-lo alterando apenas o número da versão:

```python
# Versão 1 do modelo
model = mlflow.sklearn.load_model("models:/churn model teste/1")

# Versão 2 do mesmo modelo
model = mlflow.sklearn.load_model("models:/churn model teste/2")
```

----------

#### Utilizando aliases para gerenciar o modelo de produção

Uma prática recomendada no MLflow é o uso de **aliases** para controlar qual versão do modelo está em produção, sem precisar alterar o código da aplicação.  
Em vez de carregar diretamente uma versão fixa (`models:/churn model teste/1` ou `/2`), podemos atribuir um **alias** (por exemplo, `@production`) a uma versão específica.  

Assim, o cliente sempre consome o alias `@production`, e quando for necessário promover uma nova versão do modelo, basta atualizar o alias no **Model Registry** — sem modificar nenhuma linha de código do consumidor.


In [41]:
# importar 
from mlflow.tracking import MlflowClient

# local na rede do mlflow
mlflow.set_tracking_uri("http://localhost:5000")

# importar client
client = MlflowClient()

# Aponta "production" para a versão 2 do modelo
client.set_registered_model_alias("churn model teste", "production", 1)

# Agora o consumo pode ser feito assim:
model = mlflow.sklearn.load_model("models:/churn model teste@production")


#### Visualizando todas as versões registradas de um modelo

O MLflow permite consultar facilmente todas as versões já registradas de um modelo no **Model Registry**.  
Dessa forma, podemos acompanhar o histórico completo de treinamentos, identificar de qual run cada versão foi gerada, além de verificar o status e os aliases atribuídos.


In [42]:
# importar client
client = MlflowClient()

# Lista todas as versões do modelo pelo nome
versions = client.search_model_versions("name='churn model teste'")

for v in versions:
    print(f"Versão: {v.version} | Run ID: {v.run_id} | Status: {v.status} | Source: {v.source}")


# Encontra a versão mais recente pelo número
latest_version = max(int(v.version) for v in versions)

print("Última versão registrada:", latest_version)

Versão: 2 | Run ID:  | Status: READY | Source: mlflow-artifacts:/138472396392580619/models/m-0b458a3319854e0b94765855cce3b26c/artifacts
Versão: 1 | Run ID:  | Status: READY | Source: mlflow-artifacts:/138472396392580619/models/m-7503ec581fbe4b8eb1b57389ac01c56a/artifacts
Última versão registrada: 2


#### Para exemplificar irei rodar um modelo ensemble e registrar 

In [43]:
# ---------------------------------------
# importar o VotingClassifier ensemble
from sklearn.ensemble import VotingClassifier

# ---------------------------------------

# definir o nome do experimento que iremos enviar as informações
mlflow.set_experiment("meu_experimento")

# ---------------------------------------
# Carregar dados
df = pd.read_csv("data/abt.csv", sep=",")

# Definir variáveis explicativas (usa suas colunas do 3º ao penúltimo)
features = df.columns[2:-1]
X = df[features]

# Definir variável alvo
target = "flag_churn"
y = df[target]

# Split treino/teste
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Taxa de resposta train:", y_train.mean())
print("Taxa de resposta test :", y_test.mean())

# ---------------------------------------
# Modelos base
dt = tree.DecisionTreeClassifier(
    min_samples_leaf=100,
    min_samples_split=10,
    random_state=42
)

rf = ensemble.RandomForestClassifier(
    n_estimators=500,
    min_samples_leaf=20,
    random_state=123,
    n_jobs=-1
)

# Ensemble por votação (hard vote)
voter = VotingClassifier(
    estimators=[("dt", dt), ("rf", rf)],
    voting="hard"  # para “soft”, ambos precisam de predict_proba confiável
)

# ---------------------------------------
# Treinar + logar no MLflow
mlflow.sklearn.autolog()  # captura automática (parâmetros, métricas padrão e modelo)

with mlflow.start_run(run_name="ensemble_dt_rf"):
    # info útil como tags/params extras
    mlflow.set_tag("use_case", "churn")
    mlflow.log_param("n_features", X.shape[1])
    mlflow.log_param("feature_cols_sample", ", ".join(features[:10]))  # amostra de colunas
    mlflow.log_metric("train_rate", float(y_train.mean()))
    mlflow.log_metric("test_rate", float(y_test.mean()))

    # treino
    voter.fit(X_train, y_train)

    # previsões
    y_tr_pred = voter.predict(X_train)
    y_te_pred = voter.predict(X_test)

    # métricas principais
    acc_tr = metrics.accuracy_score(y_train, y_tr_pred)
    acc_te = metrics.accuracy_score(y_test, y_te_pred)
    f1_te  = metrics.f1_score(y_test, y_te_pred, average="binary")

    # log explícito das métricas (além do autolog)
    mlflow.log_metrics({
        "acc_train": acc_tr,
        "acc_test":  acc_te,
        "f1_test":   f1_te
    })


    # salvar modelo do ensemble como artefato “model”
    mlflow.sklearn.log_model(voter, artifact_path="model")

print(f"Acurácia (train): {acc_tr:.4f}")
print(f"Acurácia (test) : {acc_te:.4f}")
print(f"F1 (test)       : {f1_te:.4f}")



Taxa de resposta train: 0.4343403205918619
Taxa de resposta test : 0.43403205918618987


2025/08/21 23:16:18 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ecosi\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/21 23:16:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ecosi\anaconda3\Lib\site-packages\mlflow\types

🏃 View run ensemble_dt_rf at: http://localhost:5000/#/experiments/138472396392580619/runs/6b683d4549044315a1b04aa2ce085a7d
🧪 View experiment at: http://localhost:5000/#/experiments/138472396392580619
Acurácia (train): 0.7411
Acurácia (test) : 0.7176
F1 (test)       : 0.6567


#### Visualizar a última versão registrada do modelo

In [44]:
# importar client
client = MlflowClient()

# Lista todas as versões do modelo pelo nome
versions = client.search_model_versions("name='churn model teste'")

for v in versions:
    print(f"Versão: {v.version} | Run ID: {v.run_id} | Status: {v.status} | Source: {v.source}")


# Encontra a versão mais recente pelo número
latest_version = max(int(v.version) for v in versions)

print("Última versão registrada:", latest_version)

Versão: 2 | Run ID:  | Status: READY | Source: mlflow-artifacts:/138472396392580619/models/m-0b458a3319854e0b94765855cce3b26c/artifacts
Versão: 1 | Run ID:  | Status: READY | Source: mlflow-artifacts:/138472396392580619/models/m-7503ec581fbe4b8eb1b57389ac01c56a/artifacts
Última versão registrada: 2


#### Outras features do MLflow

* É possivel Armazenar scripts e qualquer outro tipo de arquivos como artefatos: 
* O MLflow permite registrar artefatos junto com um experimento/execução (run).
* Você pode salvar os scripts Python (ou notebooks, arquivos de configuração, etc.) que geraram o modelo com algo como:


In [46]:

with mlflow.start_run():
    # salva um script como artefato
    mlflow.log_artifact("Notebook - Machine learning com mlflow.ipynb")


🏃 View run bouncy-asp-405 at: http://localhost:5000/#/experiments/138472396392580619/runs/d1888758279d4ba48b5f077e6b30b36b
🧪 View experiment at: http://localhost:5000/#/experiments/138472396392580619
